In [5]:
%load_ext autoreload
%autoreload 2

import shap
from desdeo_problem.problem import DiscreteDataProblem
from desdeo_tools.scalarization import SimpleASF, DiscreteScalarizer
from desdeo_tools.solver import DiscreteMinimizer
from shapley_values.explanations import why_worst, why_best, why_objective_i, largest_conflict, how_to_improve_objective_i
import pandas as pd
import numpy as np

df = pd.read_csv("../data/DTLZ2_5x_3f.csv")
pareto_f = df.to_numpy()

ideal = np.min(pareto_f[:, 0:3], axis=0)
nadir = np.max(pareto_f[:, 0:3], axis=0)

problem = DiscreteDataProblem(df, ["x1", "x2", "x3", "x4", "x5"], ["f1", "f2", "f3"], nadir, ideal)

asf = SimpleASF(np.array([1,1,1]))



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
def black_box(ref_points, problem=problem, asf=asf):
    res = np.zeros(ref_points.shape)
    for (i, ref_point) in enumerate(ref_points):
        scalarizer = DiscreteScalarizer(asf, scalarizer_args={"reference_point": np.atleast_2d(ref_point)})
        solver = DiscreteMinimizer(scalarizer)
        index = solver.minimize(problem.objectives)["x"]
        
        res[i] = problem.objectives[index]
    return res

In [7]:
missing_data = np.hstack(tuple(np.random.uniform(low=ideal[i], high=nadir[i], size=(200,1)) for i in range(ideal.shape[0])))

explainer = shap.KernelExplainer(black_box, missing_data)

Using 200 background data samples could cause slower run times. Consider using shap.sample(data, K) or shap.kmeans(data, K) to summarize the background as K samples.


In [17]:
ref_point = np.array([0.65, 0.44, 0.66])
actual = black_box(np.atleast_2d(ref_point))
shap_values = np.array(explainer.shap_values(ref_point))

In [18]:
why_worst(shap_values, ref_point, actual)

('All objectives were improved compared to the desired value.', -1, -1)

In [19]:
how_to_improve_objective_i(shap_values, 0)

('Since objective 1 and objective 2 are in great conflict, try worsening the value of objective 2 for a better result for objective 1.',
 -1,
 1)

In [21]:
new_ref_point = np.array([0.65, 0.44, 1.1*0.66])
new_actual = black_box(np.atleast_2d(new_ref_point))

print(f"Original: {actual}")
print(f"After suggestion: {new_actual}")

Original: [[0.63796297 0.40456098 0.65612618]]
After suggestion: [[0.60579285 0.40380867 0.68682181]]
